The KJV version of the bible can be retrieved from Kaggle. The other can be retrieved here: https://www.biblesupersearch.com/bible-downloads/, though some adjustments to the CSV may be required.

In [2]:
import kagglehub

# Download latest version and save in the current folder
path = kagglehub.dataset_download("phyred23/bibleverses", path="bible_data_set.csv")

# This will save the dataset in an obscure folder. You will need to find it and move it to the current folder
print("Path to dataset files:", path)

/Users/sethbrock/Desktop/IS693R/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.41M/1.41M [00:00<00:00, 25.6MB/s]

Extracting zip of bible_data_set.csv...
Path to dataset files: /Users/sethbrock/.cache/kagglehub/datasets/phyred23/bibleverses/versions/1/bible_data_set.csv


In [16]:
import pandas as pd

# Load KJV and modern translations
kjv_df = pd.read_csv("data/kjv.csv")
modern_df = pd.read_csv("data/web.csv")

# Merge the two dataframes on book, chapter, and verse
merged_df = pd.merge(kjv_df, modern_df, on=["book", "chapter", "verse"], suffixes=("_kjv", "_modern"))

# Select and rename the text columns
merged_df = merged_df[["text_kjv", "text_modern"]]
merged_df.columns = ["kjv_text", "modern_text"]

merged_df.to_csv("data/web_to_kjv.csv", index=False)


In [17]:
merged_df = pd.read_csv("data/web_to_kjv.csv")
kjv_df = pd.read_csv("data/kjv.csv")
modern_df = pd.read_csv("data/web.csv")

print(kjv_df.shape[0], "KJV verses")
print(modern_df.shape[0], "WEB verses")
print(merged_df.shape[0], "merged verses")

31102 KJV verses
31103 WEB verses
31100 merged verses
